# Food Rankings

## Description

This notebook looks at the entire USDA food database and computes a shopping cart that satisfies the specified nutritional requirements while optimizing some parameter, such as minimizing calories and carbs.

Next it analyzes each element of the shopping cart and determines which nutrient(s) it's critical for -- i.e., which nutrients would fall below specified minimums if this item were removed from the shopping cart. It saves this analysis in a database table.

Next it removes *all* of those items from the set of allowed foods, and repeats the above steps. This classifies foods into tiers, where being in tier N means that this food is part of the optimal solution if all the tiers below N are removed.

This is intended as a first step toward assessing the flexibility of our food supply for meeting nutritional requirements. By definition, tier N+1 cannot do any better than tier N at optimizing the objective function (e.g., minimizing carbs), but the question becomes: how much worse is it? How quickly does the objective function worsen as N increases?

There are other ways to approach a ranking of foods, so this is an exploratory analysis.

## Calculations

### Configuration

This step loads a notebook named `Config.ipynb`, which in turn loads other notebooks, in order to define functions and settings needed by the rest of this notebook.

In [1031]:
%run ./work/lib/Config.ipynb

### Data Initialization and Cleanup

The next few steps load food data and other information needed by the rest of the noetbook, as well as doing a little bit of cleanup on that data.

#### Foods and their Nutrient Content

In [1034]:
food_data = fetch_food_percent_rdi(age=AGE, sex=SEX)
food_data = drop_foods_with_missing_nutrient_data(food_data)

### Setting up the Optimization Problem

The following sections define the nutrition optimization problem and its constraints.

#### The Objective Function

The standard objective function here is to minimize carbohydrates. Other possibilities are to minimize calories, to 

#### The Objective Function and Initial Constraints

The standard objective function here is to minimize carbohydrates. Other possibilities are to minimize calories, to maximize fiber, etc.

The initial constraints are simply that the solution must provide the RDI of all nutrients. This means one constraint per nutrient, such that the total for that nutrient must be at least 1.00, or 100%.

In [1033]:
def optimization_problem(data, nutrient='chocdf', maximize=False):
    # Minimize Carbs
    objective = define_objective_function(data, nutrient=nutrient, maximize=maximize)

    # Satisfy the RDI of all nutrients
    constraints = initialize_constraints_with_rdi(data)

    # Set upper and lower limits on calories
    constraints = add_constraint(data, constraints, 'enerc_kcal', min=1800, max=2000)

    return objective, constraints


#### Upper Limits on Nutrient Intake

The next step loads tolerable upper limits for nutrients, if known. The nutrients are identified with a prefix of `pct_`, where it should be born in mind that this is actually the decimal value rather than a true percent. We do this in order to keep the calculations simple, but it means that `1.0` is actually 100%, and a value like `60.0` means that sixty times the RDI is seen as a tolerable upper limit.

In [1032]:
# Start with the tolerable upper limits, if known, for nutrients
upper_limits = fetch_nutrient_tolerable_upper_limits(age=AGE, sex=SEX)

# Identify nutrients that don't have tolerable upper limits defined
nutrients = np.array([elt for elt in food_data.columns if elt[:4] == 'pct_'])
nutrients = np.setdiff1d(nutrients, upper_limits['tagname'].to_numpy())

# Set a loose upper limit for those
for nutrient in nutrients:
    upper_limits.loc[len(upper_limits)] = [nutrient, 10]

# Actually turn those into constraints, which depend on the food data we're
# using.
def upper_limit_constraints(data, constraints):
    for i in range(len(upper_limits)):
        nutrient, amount = upper_limits.loc[i]
        constraints = add_constraint(food_data, constraints, nutrient, max=amount)
    
    return constraints


### Solve the Problem

In [1035]:
all_food_data = food_data
results = None
rations = None

# Solve repeatedly, printing a summary of the results
while True:
    if results is not None and results['Carb Cal'].iloc[-1] > 400:
        break

    objective, constraints = optimization_problem(food_data)
    constraints = upper_limit_constraints(food_data, constraints)

    try:
        lp_bounds = constraints.iloc[-1].values
        lp_constraints = constraints.iloc[:-1].T.values
        result = linprog(objective, A_ub=lp_constraints, b_ub=lp_bounds, options={"disp": False})
        summary = pd.concat([get_macros(food_data, result), get_micros(food_data, result)], axis=0)
 
        if results is None:
            counter = pd.Series({'Value': 1}, name="Run")
            results = summary.transpose()
        else:
            counter = pd.Series({'Value': len(results)}, name="Run")
            results = pd.concat([results, summary.transpose()], axis=0).reset_index(drop=True)

        if rations is None:
            rations = get_rations(food_data, result, truncate=True)
        else:
            rations = pd.concat([rations, get_rations(food_data, result, truncate=True)], axis=0).reset_index(drop=True)

        food_data = delete_food_items(food_data, get_rations(food_data, result))
        #food_data.reset_index(drop=True, inplace=True)
    except:
        raise
        break


In [1040]:
low_carb_foods = all_food_data[~all_food_data.isin(food_data)].dropna()
low_carb_foods.reset_index(drop=True, inplace=True)
pd.set_option('display.max_rows', None)
low_carb_foods

,food_code,main_food_description,age_from,sex,type,enerc_kcal,fat,f18d2,procnt,chocdf,...,pct_vitb6a,pct_fol,pct_choln,pct_vitb12,pct_vita_rae,pct_tocpha,pct_vitd,pct_vitk1,pct_f18d2,pct_f18d3
0,91580000.0,"Gelatin, frozen, whipped, on a stick",51.0,Male,recommended,62.0,0.00,0.000,1.22,14.19,...,0.000000,0.0025,0.000909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,91601000.0,"Ice, fruit",51.0,Male,recommended,128.0,0.00,0.000,0.40,32.60,...,0.000588,0.0000,0.007273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,91611000.0,Ice pop,51.0,Male,recommended,79.0,0.24,0.015,0.00,19.23,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001071,0.000000
3,91611050.0,"Ice pop filled with ice cream, all flavor vari...",51.0,Male,recommended,121.0,3.79,0.101,1.16,20.67,...,0.009412,0.0050,0.015636,0.054167,0.043333,0.006667,0.006667,0.000833,0.007214,0.036250
4,91611100.0,"Ice pop, sweetened with low calorie sweetener",51.0,Male,recommended,24.0,0.00,0.000,0.00,5.92,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,91621000.0,Snow cone,51.0,Male,recommended,128.0,0.00,0.000,0.40,32.60,...,0.000588,0.0000,0.007273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,91700010.0,"Candy, NFS",51.0,Male,recommended,394.0,0.20,0.000,0.00,98.00,...,0.001765,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,91700500.0,M&M's Almond Chocolate Candies,51.0,Male,recommended,522.0,27.76,3.741,7.53,60.50,...,0.032353,0.0300,0.068545,0.091667,0.026667,0.634667,0.000000,0.016667,0.267214,0.007500
8,91701010.0,"Almonds, chocolate covered",51.0,Male,recommended,574.0,45.66,9.021,17.27,30.89,...,0.053529,0.0950,0.091455,0.091667,0.020000,1.118667,0.000000,0.014167,0.644357,0.024375
9,91701020.0,"Almonds, sugar-coated",51.0,Male,recommended,474.0,17.93,3.904,10.00,68.26,...,0.035294,0.0375,0.047273,0.000000,0.000000,0.824000,0.000000,0.000000,0.278857,0.000000


In [1042]:
get_rations(food_data, result)

,food_code,main_food_description,age_from,sex,type,amount,enerc_kcal,fat,f18d2,procnt,...,pct_vitb6a,pct_fol,pct_choln,pct_vitb12,pct_vita_rae,pct_tocpha,pct_vitd,pct_vitk1,pct_f18d2,pct_f18d3
27,11320000.0,Soy milk,51.0,Male,recommended,21.034997,9.045049,0.309214,0.122844,0.546910,...,0.003836,0.004733,0.009026,0.074499,0.012855,0.001543,0.015426,0.005259,0.008775,0.009860
739,22311010.0,"Ham, smoked or cured, cooked, lean and fat eaten",51.0,Male,recommended,67.532000,100.622680,4.524644,0.555788,13.567179,...,0.138242,0.003377,0.098597,0.157575,0.004502,0.012156,0.036017,0.000000,0.039699,0.052337
776,23107000.0,"Lamb, shoulder chop, cooked, NS as to fat eaten",51.0,Male,recommended,125.928788,345.044878,24.933900,1.485960,28.107305,...,0.095558,0.066113,0.199654,1.374723,0.000000,0.011753,0.008395,0.048273,0.106140,0.265238
1052,26100123.0,"Fish, NS as to type, baked or broiled, made wi...",51.0,Male,recommended,109.720883,150.317610,4.937440,0.318191,25.038306,...,0.255585,0.027430,0.185528,1.554379,0.021944,0.046083,0.512031,0.010058,0.022728,0.036345
1125,26115123.0,"Flounder, baked or broiled, made without fat",51.0,Male,recommended,20.921158,18.410619,0.508384,0.011925,3.269977,...,0.013783,0.003138,0.031154,0.111580,0.002557,0.011158,0.048816,0.000174,0.000852,0.002877
1236,26141180.0,"Sea bass, pickled",51.0,Male,recommended,31.048052,48.124481,3.185530,0.282227,3.418391,...,0.044198,0.006210,0.020774,0.019405,0.007590,0.037051,0.066236,0.014748,0.020159,0.013778
4865,63219110.0,"Raspberries, cooked or canned, NS as to sweete...",51.0,Male,recommended,15.983808,14.545266,0.019181,0.007193,0.132666,...,0.003949,0.004396,0.002383,0.000000,0.000355,0.006287,0.000000,0.006926,0.000514,0.002298
5292,72122221.0,"Mustard greens, cooked, from fresh, fat added ...",51.0,Male,recommended,152.965560,70.364157,4.298332,0.587388,3.793546,...,0.128671,0.034417,0.002503,0.000000,1.050364,0.207013,0.010198,7.277337,0.041956,0.060230
5338,72128224.0,"Turnip greens, cooked, NS as to form, made wit...",51.0,Male,recommended,36.928203,16.986973,0.897355,0.132572,1.200167,...,0.023243,0.035082,0.000604,0.000000,0.221569,0.069917,0.000000,1.546676,0.009469,0.034851
5469,73111202.0,"Peas and carrots, cooked, from frozen, NS as t...",51.0,Male,recommended,131.448933,85.441806,3.259934,0.625697,3.943468,...,0.098200,0.082156,0.042781,0.000000,0.702522,0.068353,0.000000,0.212509,0.044693,0.058330
